In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [16]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### REPEATING
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    ##################################################### REPEATING
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    ##################################################### REPEATING
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    ##################################################### REPEATING
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    ##################################################### REPEATING
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    ##################################################### REPEATING
    for _ in range(10):
        next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                       reuse=True)
        next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                    reuse=True)
        gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                             action_size=action_size, reuse=True)
        dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
        gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
        aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                         labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [17]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [19]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [20]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [21]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [22]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [23]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [24]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates >= np.max(rates)]
            actions = actions[rates >= np.max(rates)]
            next_states = next_states[rates >= np.max(rates)]
            rewards = rewards[rates >= np.max(rates)]
            dones = dones[rates >= np.max(rates)]
            rates = rates[rates >= np.max(rates)]
            aloss, dloss, _, _ = sess.run([model.a_loss, model.d_loss, model.a_opt, model.d_opt],
                                          feed_dict = {model.states: states, 
                                                       model.actions: actions,
                                                       model.next_states: next_states,
                                                       model.rewards: rewards,
                                                       model.dones: dones,
                                                       model.rates: rates})
            aloss2, _ = sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:10.0000 R:10.0000 rate:0.0200 aloss:1.3886 dloss:1.2165 aloss2:11.9103 exploreP:0.9990
Episode:1 meanR:11.0000 R:12.0000 rate:0.0240 aloss:1.3868 dloss:1.2290 aloss2:11.8712 exploreP:0.9978
Episode:2 meanR:14.3333 R:21.0000 rate:0.0420 aloss:1.3859 dloss:1.2228 aloss2:11.8780 exploreP:0.9958
Episode:3 meanR:14.5000 R:15.0000 rate:0.0300 aloss:1.3835 dloss:1.2339 aloss2:11.8590 exploreP:0.9943
Episode:4 meanR:20.8000 R:46.0000 rate:0.0920 aloss:1.3828 dloss:1.2084 aloss2:11.8764 exploreP:0.9898
Episode:5 meanR:19.3333 R:12.0000 rate:0.0240 aloss:1.3830 dloss:1.1964 aloss2:11.8858 exploreP:0.9886
Episode:6 meanR:18.8571 R:16.0000 rate:0.0320 aloss:1.3800 dloss:1.1917 aloss2:11.8849 exploreP:0.9870
Episode:7 meanR:20.1250 R:29.0000 rate:0.0580 aloss:1.3798 dloss:1.1880 aloss2:11.8837 exploreP:0.9842
Episode:8 meanR:21.1111 R:29.0000 rate:0.0580 aloss:1.3775 dloss:1.1651 aloss2:11.9043 exploreP:0.9814
Episode:9 meanR:20.9000 R:19.0000 rate:0.0380 aloss:1.3830 dloss:1.1452 a

Episode:79 meanR:20.6250 R:10.0000 rate:0.0200 aloss:1.3969 dloss:0.5951 aloss2:13.1936 exploreP:0.8494
Episode:80 meanR:20.5432 R:14.0000 rate:0.0280 aloss:1.3877 dloss:0.6004 aloss2:13.1897 exploreP:0.8482
Episode:81 meanR:20.7073 R:34.0000 rate:0.0680 aloss:1.4030 dloss:0.5934 aloss2:13.3280 exploreP:0.8454
Episode:82 meanR:20.5663 R:9.0000 rate:0.0180 aloss:1.3966 dloss:0.6223 aloss2:13.2294 exploreP:0.8446
Episode:83 meanR:20.4643 R:12.0000 rate:0.0240 aloss:1.4013 dloss:0.6085 aloss2:13.2170 exploreP:0.8436
Episode:84 meanR:20.3882 R:14.0000 rate:0.0280 aloss:1.3920 dloss:0.5705 aloss2:13.2754 exploreP:0.8425
Episode:85 meanR:20.3488 R:17.0000 rate:0.0340 aloss:1.3866 dloss:0.5853 aloss2:13.2684 exploreP:0.8411
Episode:86 meanR:20.4483 R:29.0000 rate:0.0580 aloss:1.3925 dloss:0.5858 aloss2:13.3621 exploreP:0.8387
Episode:87 meanR:20.3750 R:14.0000 rate:0.0280 aloss:1.4360 dloss:0.5442 aloss2:13.5548 exploreP:0.8375
Episode:88 meanR:20.3820 R:21.0000 rate:0.0420 aloss:1.3980 dloss

Episode:158 meanR:24.0200 R:43.0000 rate:0.0860 aloss:1.4345 dloss:0.4791 aloss2:14.1289 exploreP:0.6968
Episode:159 meanR:23.9800 R:19.0000 rate:0.0380 aloss:1.4250 dloss:0.4952 aloss2:14.1217 exploreP:0.6955
Episode:160 meanR:24.3200 R:43.0000 rate:0.0860 aloss:1.4124 dloss:0.5127 aloss2:14.1139 exploreP:0.6925
Episode:161 meanR:24.5900 R:38.0000 rate:0.0760 aloss:1.4201 dloss:0.4813 aloss2:14.1004 exploreP:0.6899
Episode:162 meanR:25.2400 R:89.0000 rate:0.1780 aloss:1.4261 dloss:0.5040 aloss2:14.1417 exploreP:0.6839
Episode:163 meanR:25.2700 R:37.0000 rate:0.0740 aloss:1.4123 dloss:0.4839 aloss2:14.1682 exploreP:0.6814
Episode:164 meanR:25.5100 R:40.0000 rate:0.0800 aloss:1.4373 dloss:0.4607 aloss2:14.2510 exploreP:0.6787
Episode:165 meanR:25.7600 R:38.0000 rate:0.0760 aloss:1.4224 dloss:0.4704 aloss2:14.1433 exploreP:0.6762
Episode:166 meanR:25.7200 R:38.0000 rate:0.0760 aloss:1.4296 dloss:0.4701 aloss2:14.2012 exploreP:0.6737
Episode:167 meanR:26.2200 R:79.0000 rate:0.1580 aloss:1

Episode:236 meanR:52.5900 R:104.0000 rate:0.2080 aloss:1.4524 dloss:0.4881 aloss2:13.9439 exploreP:0.4539
Episode:237 meanR:52.4600 R:23.0000 rate:0.0460 aloss:1.4224 dloss:0.5627 aloss2:13.9643 exploreP:0.4529
Episode:238 meanR:52.4900 R:53.0000 rate:0.1060 aloss:1.4196 dloss:0.5015 aloss2:13.9256 exploreP:0.4505
Episode:239 meanR:53.2700 R:118.0000 rate:0.2360 aloss:1.4454 dloss:0.4657 aloss2:14.1282 exploreP:0.4454
Episode:240 meanR:54.4400 R:180.0000 rate:0.3600 aloss:1.4094 dloss:0.4806 aloss2:14.5376 exploreP:0.4376
Episode:241 meanR:55.1300 R:91.0000 rate:0.1820 aloss:1.3902 dloss:0.4727 aloss2:14.1390 exploreP:0.4337
Episode:242 meanR:55.8800 R:140.0000 rate:0.2800 aloss:1.3930 dloss:0.4768 aloss2:13.9505 exploreP:0.4278
Episode:243 meanR:57.0400 R:145.0000 rate:0.2900 aloss:1.3992 dloss:0.4877 aloss2:13.8795 exploreP:0.4218
Episode:244 meanR:56.9600 R:28.0000 rate:0.0560 aloss:1.3936 dloss:0.5171 aloss2:13.8565 exploreP:0.4207
Episode:245 meanR:57.1800 R:64.0000 rate:0.1280 al

Episode:314 meanR:122.5100 R:129.0000 rate:0.2580 aloss:1.4678 dloss:0.6344 aloss2:13.5978 exploreP:0.1620
Episode:315 meanR:121.5300 R:89.0000 rate:0.1780 aloss:1.4586 dloss:0.6802 aloss2:13.5081 exploreP:0.1607
Episode:316 meanR:122.8800 R:181.0000 rate:0.3620 aloss:1.4390 dloss:0.6726 aloss2:13.5285 exploreP:0.1580
Episode:317 meanR:124.2000 R:195.0000 rate:0.3900 aloss:1.4563 dloss:0.6298 aloss2:13.4405 exploreP:0.1551
Episode:318 meanR:124.1500 R:57.0000 rate:0.1140 aloss:1.4342 dloss:0.7708 aloss2:13.4556 exploreP:0.1543
Episode:319 meanR:124.3200 R:57.0000 rate:0.1140 aloss:1.4308 dloss:0.6453 aloss2:13.4180 exploreP:0.1535
Episode:320 meanR:125.7800 R:203.0000 rate:0.4060 aloss:1.4520 dloss:0.6353 aloss2:13.4125 exploreP:0.1506
Episode:321 meanR:127.0100 R:229.0000 rate:0.4580 aloss:1.4529 dloss:0.6196 aloss2:117.7648 exploreP:0.1474
Episode:322 meanR:128.6500 R:188.0000 rate:0.3760 aloss:1.4051 dloss:0.6190 aloss2:323.4272 exploreP:0.1449
Episode:323 meanR:128.4300 R:37.0000 r

Episode:391 meanR:219.3600 R:403.0000 rate:0.8060 aloss:1.4034 dloss:0.8436 aloss2:12.9626 exploreP:0.0354
Episode:392 meanR:219.3600 R:97.0000 rate:0.1940 aloss:1.4051 dloss:0.8273 aloss2:12.9210 exploreP:0.0351
Episode:393 meanR:215.7300 R:115.0000 rate:0.2300 aloss:1.3972 dloss:0.8819 aloss2:13.0016 exploreP:0.0348
Episode:394 meanR:213.9300 R:123.0000 rate:0.2460 aloss:1.4004 dloss:0.9041 aloss2:12.9041 exploreP:0.0345
Episode:395 meanR:212.8800 R:99.0000 rate:0.1980 aloss:1.4093 dloss:0.8263 aloss2:12.9098 exploreP:0.0343
Episode:396 meanR:212.9200 R:146.0000 rate:0.2920 aloss:1.3997 dloss:0.8706 aloss2:12.8719 exploreP:0.0339
Episode:397 meanR:212.5600 R:112.0000 rate:0.2240 aloss:1.4042 dloss:0.8522 aloss2:12.8820 exploreP:0.0337
Episode:398 meanR:212.0200 R:82.0000 rate:0.1640 aloss:1.3938 dloss:0.8445 aloss2:12.9845 exploreP:0.0335
Episode:399 meanR:211.5100 R:120.0000 rate:0.2400 aloss:1.4047 dloss:0.8722 aloss2:12.9309 exploreP:0.0332
Episode:400 meanR:208.2300 R:100.0000 ra

Episode:468 meanR:183.0300 R:92.0000 rate:0.1840 aloss:1.3149 dloss:1.0039 aloss2:12.7484 exploreP:0.0166
Episode:469 meanR:183.6000 R:244.0000 rate:0.4880 aloss:1.3553 dloss:0.8671 aloss2:12.9370 exploreP:0.0164
Episode:470 meanR:183.6100 R:98.0000 rate:0.1960 aloss:1.3121 dloss:1.0003 aloss2:12.8876 exploreP:0.0163
Episode:471 meanR:183.6100 R:83.0000 rate:0.1660 aloss:1.3339 dloss:0.9611 aloss2:12.9145 exploreP:0.0163
Episode:472 meanR:180.0900 R:131.0000 rate:0.2620 aloss:1.3573 dloss:0.9300 aloss2:12.9312 exploreP:0.0162
Episode:473 meanR:180.5100 R:125.0000 rate:0.2500 aloss:1.3362 dloss:0.9465 aloss2:12.9259 exploreP:0.0161
Episode:474 meanR:177.9600 R:206.0000 rate:0.4120 aloss:1.3288 dloss:0.9314 aloss2:12.9222 exploreP:0.0160
Episode:475 meanR:177.2200 R:93.0000 rate:0.1860 aloss:1.3442 dloss:0.9138 aloss2:12.9356 exploreP:0.0159
Episode:476 meanR:175.4200 R:97.0000 rate:0.1940 aloss:1.3274 dloss:0.8925 aloss2:12.9101 exploreP:0.0159
Episode:477 meanR:175.5000 R:89.0000 rate:

Episode:546 meanR:133.5100 R:196.0000 rate:0.3920 aloss:1.3245 dloss:0.9489 aloss2:12.9281 exploreP:0.0129
Episode:547 meanR:132.9400 R:138.0000 rate:0.2760 aloss:1.3058 dloss:0.9574 aloss2:12.9536 exploreP:0.0129
Episode:548 meanR:131.3000 R:95.0000 rate:0.1900 aloss:1.3247 dloss:0.9955 aloss2:12.9965 exploreP:0.0128
Episode:549 meanR:130.6900 R:116.0000 rate:0.2320 aloss:1.3117 dloss:1.0295 aloss2:12.8891 exploreP:0.0128
Episode:550 meanR:130.6200 R:135.0000 rate:0.2700 aloss:1.3309 dloss:0.9971 aloss2:12.9122 exploreP:0.0128
Episode:551 meanR:129.9700 R:113.0000 rate:0.2260 aloss:1.3186 dloss:0.8957 aloss2:12.9419 exploreP:0.0127
Episode:552 meanR:128.2700 R:75.0000 rate:0.1500 aloss:1.3222 dloss:0.8881 aloss2:12.9981 exploreP:0.0127
Episode:553 meanR:127.1800 R:92.0000 rate:0.1840 aloss:1.3089 dloss:0.9443 aloss2:12.9767 exploreP:0.0127
Episode:554 meanR:126.2900 R:128.0000 rate:0.2560 aloss:1.3038 dloss:0.9627 aloss2:12.9200 exploreP:0.0127
Episode:555 meanR:126.3100 R:193.0000 ra

Episode:623 meanR:122.6500 R:205.0000 rate:0.4100 aloss:1.2452 dloss:1.0173 aloss2:12.7728 exploreP:0.0111
Episode:624 meanR:123.2500 R:125.0000 rate:0.2500 aloss:1.2736 dloss:0.9924 aloss2:12.8224 exploreP:0.0111
Episode:625 meanR:123.3500 R:70.0000 rate:0.1400 aloss:1.2856 dloss:0.9760 aloss2:12.8224 exploreP:0.0111
Episode:626 meanR:123.9600 R:159.0000 rate:0.3180 aloss:1.2407 dloss:1.0431 aloss2:12.7870 exploreP:0.0110
Episode:627 meanR:124.4700 R:127.0000 rate:0.2540 aloss:1.2313 dloss:1.0483 aloss2:12.7543 exploreP:0.0110
Episode:628 meanR:125.6000 R:168.0000 rate:0.3360 aloss:1.2658 dloss:1.0432 aloss2:12.7662 exploreP:0.0110
Episode:629 meanR:124.4900 R:162.0000 rate:0.3240 aloss:1.2662 dloss:1.0242 aloss2:12.7689 exploreP:0.0110
Episode:630 meanR:124.7100 R:133.0000 rate:0.2660 aloss:1.2584 dloss:1.0141 aloss2:12.7546 exploreP:0.0110
Episode:631 meanR:125.3300 R:126.0000 rate:0.2520 aloss:1.2391 dloss:1.0145 aloss2:12.7879 exploreP:0.0110
Episode:632 meanR:126.0400 R:170.0000 

Episode:700 meanR:141.0100 R:157.0000 rate:0.3140 aloss:1.2244 dloss:1.0432 aloss2:12.7015 exploreP:0.0104
Episode:701 meanR:141.5300 R:138.0000 rate:0.2760 aloss:1.2095 dloss:1.0542 aloss2:12.7030 exploreP:0.0103
Episode:702 meanR:142.3100 R:159.0000 rate:0.3180 aloss:1.2375 dloss:1.0073 aloss2:12.7712 exploreP:0.0103
Episode:703 meanR:143.7900 R:217.0000 rate:0.4340 aloss:1.2102 dloss:1.0553 aloss2:12.7255 exploreP:0.0103
Episode:704 meanR:144.1900 R:202.0000 rate:0.4040 aloss:1.2141 dloss:1.0194 aloss2:12.7581 exploreP:0.0103
Episode:705 meanR:144.5000 R:179.0000 rate:0.3580 aloss:1.1925 dloss:1.0861 aloss2:12.7044 exploreP:0.0103
Episode:706 meanR:145.6000 R:214.0000 rate:0.4280 aloss:1.2040 dloss:1.0417 aloss2:12.7137 exploreP:0.0103
Episode:707 meanR:146.2800 R:153.0000 rate:0.3060 aloss:1.2104 dloss:1.0756 aloss2:12.6937 exploreP:0.0103
Episode:708 meanR:146.0900 R:122.0000 rate:0.2440 aloss:1.1979 dloss:1.0450 aloss2:12.6930 exploreP:0.0103
Episode:709 meanR:146.2900 R:144.0000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.